In [1]:
import os
import leafmap.leafmap as leafmap

import leafmap.colormaps as cm
from samgeo import SamGeo2
from samgeo.common import regularize
import tifffile

import rasterio
import geopandas as gpd

In [2]:
image_path = os.path.join("GISTDA","TH2_PATTYA","PANSHAPEN", "T2_PRO_ISP202601200442250549_20260120044523229_2", "VOL_T2V_ORTHO_DS_T2V_202501190343234_VXB_E101N13_007823_32", "IMG_T2V_ORTHO_PMS_32", "IMG_T2V_20250119034323_ORTHO_PMS_32.TIF")

In [3]:
img = tifffile.imread(image_path)

raster_src = rasterio.open(image_path)
print(f"Raster CRS: {raster_src.crs}")


Raster CRS: EPSG:32647


In [4]:
m = leafmap.Map()  
m.add_raster(image_path, layer_name="Theos" )  
m

Map(center=[13.000913, 100.9426325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [5]:
m.user_rois

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[100.924079, 13.050403],
      [100.924079, 13.053288],
      [100.929068, 13.053288],
      [100.929068, 13.050403],
      [100.924079, 13.050403]]]}}]}

In [6]:
top_left_coordinates = [100.883503, 13.058409] # long, lat 
bottom_right_coordinates = [101.001263, 12.94316] # long, lat 

image_height = img.shape[0]
image_width = img.shape[1]


def latlng_to_pixel(long, lat):
    lat      = lat

    min_long = min(top_left_coordinates[0], bottom_right_coordinates[0])
    max_long = max(top_left_coordinates[0], bottom_right_coordinates[0])

    min_lat  = min(top_left_coordinates[1], bottom_right_coordinates[1])
    max_lat  = max(top_left_coordinates[1], bottom_right_coordinates[1])
    pixel_x =  ((long - min_long) / (max_long - min_long))*image_width  
    pixel_y =  (1-(lat - min_lat) / (max_lat - min_lat))*image_height  
    pixel = [int(pixel_x), int(pixel_y)]
    return pixel

In [7]:
coordinates = m.user_rois['features'][0]['geometry']['coordinates'][0]

In [8]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32647", always_xy=True)
poly_gons = []
for coord in coordinates: 
    easting, northing = transformer.transform(coord[0], coord[1])
    poly_gons.append([easting, northing])

In [9]:
poly_gons

[[708643.017080186, 1443500.897371503],
 [708640.5953636374, 1443820.1026299284],
 [709181.7702060998, 1443824.2141380198],
 [709184.1982132446, 1443505.0080344349],
 [708643.017080186, 1443500.897371503]]

In [10]:
clipped_image_file = "clipped_image.tif"
leafmap.clip_image(image_path, poly_gons, clipped_image_file)

Reading input: /mnt/HDD2/GISTDA_building_detection/clipped_image.tif

Adding overviews...
Updating dataset tags...
Writing output to: /mnt/HDD2/GISTDA_building_detection/clipped_image.tif


In [11]:
import cv2
import numpy as np
clipped_image = tifffile.imread(clipped_image_file) 
normalized_img = cv2.normalize(clipped_image, None, 0, 255, cv2.NORM_MINMAX)  # Normalize to 0-1 range
final_img = normalized_img.astype(np.uint8) 
tifffile.imwrite('equalized_clipped_image.tif', final_img)
# 7. Save the result as TIFF



In [12]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False
)

In [15]:
sam.set_image("clipped_image.tif")

[ WARN:0@95.202] global grfmt_tiff.cpp:122 TIFF_Warning TIFFReadDirectory: Unknown field with tag 33550 (0x830e) encountered
[ WARN:0@95.202] global grfmt_tiff.cpp:122 TIFF_Warning TIFFReadDirectory: Unknown field with tag 33922 (0x8482) encountered
[ WARN:0@95.202] global grfmt_tiff.cpp:122 TIFF_Warning TIFFReadDirectory: Unknown field with tag 34735 (0x87af) encountered
[ WARN:0@95.202] global grfmt_tiff.cpp:122 TIFF_Warning TIFFReadDirectory: Unknown field with tag 34737 (0x87b1) encountered
[ WARN:0@95.202] global grfmt_tiff.cpp:122 TIFF_Warning TIFFReadDirectory: Unknown field with tag 42112 (0xa480) encountered


In [ ]:
sam.show_map()